**Enfoque del Script:** Concatenación de dataframes (1x muestra) y fusión de todos ellos en un solo dataframe sin duplicados mediante group by y con conteo de muestras que contienen ese SNP.

In [ ]:
import pandas as pd
import os
import glob
from datetime import datetime

main_directory = "/home/laura/andrea/scripts/Samples_Script_Test"
output_directory = "/home/laura/andrea/scripts/Filtered_SNPs_TB_DDBB/Script_2"

DDBB_file = "/home/laura/andrea/scripts/Filtered_SNPs_TB_DDBB/Script_1/Filtered_SNPs_20250220_135355.tsv"

# DDBB en dataframe
df_filtered_SNPs = pd.read_csv(DDBB_file, sep='\t')

**Filtrado de SNPS:**

In [6]:
df_files = None

# Búsqueda de archivos TSV en todas las subcarpetas:
for folder_path, subfolders, files in os.walk(main_directory):

    if os.path.isdir(folder_path):
        tsv_files = glob.glob(os.path.join(folder_path, "snps.all.ivar.tsv"))  # Busca archivos en cada subcarpeta con el patrón especificado

        for file in tsv_files:
            try:
                df_file_sample = pd.read_csv(file, sep='\t')

                sample_name = os.path.basename(os.path.dirname(file))
                df_file_sample["SAMPLES"] = sample_name

                # Filtros quality e INDELs:
                df_file_sample = df_file_sample[(df_file_sample['TOTAL_DP'] >= 20) & (df_file_sample['ALT_FREQ'] >= 0.7)]
                        
                df_file_sample = df_file_sample[(df_file_sample['TYPE'] == "snp")]

                if df_files is None:
                    df_files = df_file_sample

                else:
                    df_files = pd.concat([df_files, df_file_sample], ignore_index = True)
            
            except Exception as e:
                print(f'Error procesando {file}: {e}')

# Fusionar filas con el mismo POS, REF y ALT:
df_files = df_files.groupby(['POS', 'REF', 'ALT'], as_index=False).agg({'SAMPLES': lambda x: ', '.join(sorted(set(x)))}) # Unir nombres de muestras sin duplicados
     
df_files['COUNT'] = df_files['SAMPLES'].apply(lambda x: len(x.split(", ")))

print(df_files)

timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

output_file = f'/home/laura/andrea/scripts/Filtered_SNPs_TB_DDBB/Script_2/Filtered_SNPs_fusionados_{timestamp}.tsv'
df_files.to_csv(output_file, sep='\t', index=False)

          POS REF ALT                                            SAMPLES  \
0          29   C   G                                        SRR30362894   
1        1179   G   C                                        SRR30362894   
2        2005   G   A                                        SRR30362893   
3        2532   C   T  SRR30362888, SRR30362889, SRR30362890, SRR3036...   
4        4771   A   G                                        SRR30362895   
...       ...  ..  ..                                                ...   
2981  4408156   A   C                                        SRR30362895   
2982  4408168   A   G                                        SRR30362888   
2983  4408920   G   A  SRR30362888, SRR30362889, SRR30362890, SRR3036...   
2984  4408923   T   C  SRR30362888, SRR30362889, SRR30362890, SRR3036...   
2985  4410913   C   T              SRR30362890, SRR30362893, SRR30362897   

      COUNT  
0         1  
1         1  
2         1  
3        10  
4         1  
...